In [ ]:
%load_ext snakeviz
%load_ext autoreload
%autoreload 2
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
from scipy import sparse
from letitbee import *
from particle import *
from bayes import *
from probutils import *
from audioutils import *
import time

# Let It Bee

In [ ]:
sr = 44100
win = 2048
hop = win//2
p = 10
stereo = True

#ytarget, sr = librosa.load("Snarkcharm-2-fullmixloop.wav", sr=sr, mono=not stereo)
#ycorpus = load_corpus("EdenVIP2", sr, True)

#ytarget, sr = librosa.load("ParadisoBlip2.wav", sr=sr, mono=not stereo)
#ycorpus = load_corpus("BenNMFData/PZLetItBee-main/BigOrganismCorpus", sr, True)

ytarget, sr = librosa.load("aha.mp3", sr=sr, mono=not stereo)
ycorpus = load_corpus("theoshift.mp3", sr, True)

if stereo and len(ytarget.shape) == 1:
    ytarget = np.array([ytarget, ytarget])

W1L = get_windowed(ytarget[0, :], hop, win)
W1R = get_windowed(ytarget[1, :], hop, win)
VL = np.abs(np.fft.fft(W1L, axis=0)[0:win//2+1, :])
VR = np.abs(np.fft.fft(W1R, axis=0)[0:win//2+1, :])
V = np.concatenate((VL, VR), axis=0)

WSoundL = get_windowed(ycorpus[0, :], hop, win)
WSoundR = get_windowed(ycorpus[1, :], hop, win)
WL = np.abs(np.fft.fft(WSoundL, axis=0)[0:win//2+1, :])
WR = np.abs(np.fft.fft(WSoundR, axis=0)[0:win//2+1, :])
W = np.concatenate((WL, WR), axis=0)

In [ ]:
c = 3 # Diagonal
r = 3 # Repeated activation
tic = time.time()
HGT = get_musaic_activations(V, W, L=50, r=r, p=p, c=c, verbose=False)
print("Elapsed time ground truth: {:.3f}".format(time.time()-tic))
WHGT = W.dot(HGT)
gt_fit = np.sum(V*np.log(V/WHGT) - V + WHGT)

yL = do_windowed_sum(WSoundL, HGT, win, hop)
yR = do_windowed_sum(WSoundR, HGT, win, hop)

y = np.array([yL, yR]).T
y = y/np.max(y)

ipd.Audio(y.T, rate=sr)

In [ ]:
pparticle = p
pd = 0.99
temperature = 100
L = 10
P = 2000
gamma = 0
neff_thresh = 0.1*P
tic = time.time()


pf = ParticleFilter(ycorpus=ycorpus, win=win, p=p, pfinal=p, pd=pd, temperature=temperature, L=L, P=P, gamma=gamma, r=r, neff_thresh=neff_thresh, use_gpu=True)
%snakeviz ygen = pf.process_audio_offline(ytarget)

print("Elapsed time particle filter: {:.3f}".format(time.time()-tic))

In [ ]:
H = pf.get_H()

prefix = "Particle_p{}_pd{}_temperature{}_L{}_r{}_P{}_neff{}".format(p, pd, temperature, L, r, P, neff_thresh)

#H, stats = get_bayes_musaic_activations(V, W, p, pd, temperature, L, gamma=gamma, r=r)
#prefix = "Bayes_p{}_pd{}_temperature{}_L{}_r{}".format(p, pd, temperature, L, r)
#print("Elapsed time bayes filter: {:.3f}".format(time.time()-tic))

wsmax, neff = pf.wsmax, pf.neff

a1 = get_activations_diff(HGT, p)
a2 = get_activations_diff(H, p)
r1 = get_repeated_activation_itervals(HGT, p)
r2 = get_repeated_activation_itervals(H, p)

WH = W.dot(H)
fit = np.sum(V*np.log(V/WH) - V + WH)
yL = do_windowed_sum(WSoundL, H, win, hop)
yR = do_windowed_sum(WSoundR, H, win, hop)

y = np.array([yL, yR]).T
y = y/np.max(y)

y = y/np.max(y)
wavfile.write("{}.wav".format(prefix), sr, y)

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.plot(a1)
plt.plot(a2)
plt.legend(["Ground Truth: Mean {:.3f}".format(np.mean(a1)),
            "Particle Filter: Mean {:.3f}".format(np.mean(a2))])
plt.title("Activation Changes over Time, Temperature {}".format(temperature))
plt.xlabel("Timestep")
plt.subplot(232)
plt.hist(a1, bins=np.arange(p+2), alpha=0.5)
plt.hist(a2, bins=np.arange(p+2), alpha=0.5)
plt.title("Activation Changes Histogram")
plt.xlabel("Number of Activations Changed")
plt.ylabel("Counts")
plt.legend(["Ground Truth", "Particle Filter"])
plt.subplot(233)
plt.hist(r1, bins=np.arange(c*20), alpha=0.5)
plt.hist(r2, bins=np.arange(c*20), alpha=0.5)
plt.title("Repeated Activations Histogram, r={}".format(r))
plt.xlabel("Repeated Activation Distance")
plt.ylabel("Counts")
plt.legend(["Ground Truth", "Particle Filter"])
plt.subplot(234)
plt.plot(wsmax)
plt.title("Max Probability, Overall Fit: {:.3f} ({:.3f}x)\nGround Truth Fit: {:.3f}".format(fit, fit/gt_fit, gt_fit))
plt.xlabel("Timestep")
plt.subplot(235)
plt.plot(neff)
plt.xlabel("Timestep")
plt.title("Number of Effective Particles (Median {:.2f})".format(np.median(neff)))
plt.subplot(236)
diagsgt = get_diag_lengths(HGT, p)
diags = get_diag_lengths(H, p)
plt.hist(diagsgt, bins=np.arange(30), alpha=0.5)
plt.hist(diags, bins=np.arange(30), alpha=0.5)
plt.legend(["Ground Truth Mean: {:.3f} (c={})".format(np.mean(diagsgt), c),
            "Particle Filter Mean: {:.3f} ($p_d$={})".format(np.mean(diags), pd)])
plt.xlabel("Diagonal Length")
plt.ylabel("Counts")
plt.title("Diagonal Lengths")
plt.tight_layout()
plt.savefig("{}.svg".format(prefix), bbox_inches='tight')

ipd.Audio(y.T, rate=sr)

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(311)
plt.imshow(V, cmap='magma_r', aspect='auto')
plt.title("V")
plt.colorbar()
plt.subplot(312)
plt.imshow(WHGT, cmap='magma_r', aspect='auto')
plt.title("Ground Truth")
plt.colorbar()
plt.subplot(313)
plt.imshow(WH, cmap='magma_r', aspect='auto')
plt.title("My Technique")
plt.colorbar()